# 5. Deploying Machine Learning models 

We'll use the same model we trained and evaluated previously - the churn prediction model. Now we'll deploy it as a web service.

## 5.1 Intro / Session overview
We need to put the model that lives in our Jupyter Notebook into production, so other services can use the model to make decisions based on the output of our model.

Suppose we have a service for running marketing campaigns. For each customer, it needs to determine the probability of churn, and if it's high enough, it will send a promotional email with discounts. This service needs to use our model to decide whether it should send an email. 

What we will cover this week: 
* Saving models with Pickle
* Serving models with Flask
* Managing dependencies with Pipenv
* Making the service self-contained with Docker
* Deploying it to the cloud using AWS Elastic Beanstalk

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [3]:
# df = pd.read_csv('data-week-3.csv')
df = pd.read_csv(data)

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [4]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [5]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [6]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=3000)
    model.fit(X_train, y_train)
    
    return dv, model

In [7]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [8]:
C = 1.0
n_splits = 5

In [9]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values

    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

C=1.0 0.842 +- 0.007


In [10]:
scores

[np.float64(0.8443806862337213),
 np.float64(0.8449563799496754),
 np.float64(0.83351796106763),
 np.float64(0.8347649005563726),
 np.float64(0.8517892441404411)]

In [11]:
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)

y_test = df_test.churn.values
auc = roc_auc_score(y_test, y_pred)
auc

np.float64(0.8583598751990639)

## 5.2 Saving and loading the model

* Saving the model to pickle
* Loading the model from pickle
* Turning our notebook into a Python script

To be able to use it outside of our notebook, we need to save it, and then later, another process can load and use it.

Pickle is a serialization/deserialization module that's already built into Python: using it, we can save an arbitrary Python object (with a few exceptions) to a file. Once we have a file, we can load the model from there in a different process.

Install the library with the command `pip install pickle-mixin` if you don't have it.

To save the model, we first import the `pickle` module, and then use the `dump` function:
  ```python
  import pickle

  with open('model.bin', 'wb') as f_out:  # 'wb' means write binary
      pickle.dump((dict_vectorizer, model), f_out)
  ```
  
To use the model, we need to open the binary file we saved and load the model using the `load` function.

  ```python
  import pickle

  with open('model.bin', 'rb') as f_in:  # 'rb' means read binary
      # Note: Never open a binary file from an untrusted source!
      dict_vectorizer, model = pickle.load(f_in)
  ```

#### Save the model

In [12]:
import pickle

In [13]:
output_file = f'model_C={C}.bin'

In [14]:
output_file

'model_C=1.0.bin'

In [15]:
# Write to the file in binary
f_out = open(output_file, 'wb') 
pickle.dump((dv, model), f_out)
f_out.close()

In [16]:
# !ls -lh *.bin

In [17]:
with open(output_file, 'wb') as f_out: 
    pickle.dump((dv, model), f_out)

#### Load the model

In [18]:
import pickle

In [19]:
input_file = 'model_C=1.0.bin'

Be careful when specifying the mode. Accidentally specifying an incorrect mode may result in data loss: if you open an existing file with the `w` mode instead of `r`, it will overwrite the content.

Also, unpickling objects found on the internet is not secure: it can execute arbitrary code on your machine. Use it only for things you trust and things you saved yourself.

In [20]:
with open(input_file, 'rb') as f_in: 
    dv, model = pickle.load(f_in)

In [21]:
dv, model

(DictVectorizer(sparse=False), LogisticRegression(max_iter=3000))

Notice that we did not import scikit-learn but we need to have scikit-learn installed on our computer for this to work. Otherwise, it will complain not knowing what this is (referring to these classes) when we try to load the pickle file and this is because scikit-learn is not installed on our computer. 

In [22]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [23]:
# Turn this customer into feature matrix
X = dv.transform([customer])
X

array([[ 1.  ,  0.  ,  0.  ,  1.  ,  0.  ,  1.  ,  0.  ,  0.  ,  1.  ,
         0.  ,  1.  ,  0.  ,  0.  , 29.85,  0.  ,  1.  ,  0.  ,  0.  ,
         0.  ,  1.  ,  1.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  1.  ,
         0.  ,  0.  ,  1.  ,  0.  ,  1.  ,  0.  ,  0.  ,  1.  ,  0.  ,
         0.  ,  1.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  1.  , 29.85]])

In [24]:
model.predict_proba(X)

array([[0.37255464, 0.62744536]])

In [25]:
model.predict_proba(X)[:, 1]

array([0.62744536])

In [26]:
# Get the probability that this particular customer is going to churn
y_pred = model.predict_proba(X)[0, 1]
y_pred

np.float64(0.6274453618230489)

In [27]:
print('input:', customer)
print('output:', y_pred)

input: {'gender': 'female', 'seniorcitizen': 0, 'partner': 'yes', 'dependents': 'no', 'phoneservice': 'no', 'multiplelines': 'no_phone_service', 'internetservice': 'dsl', 'onlinesecurity': 'no', 'onlinebackup': 'yes', 'deviceprotection': 'no', 'techsupport': 'no', 'streamingtv': 'no', 'streamingmovies': 'no', 'contract': 'month-to-month', 'paperlessbilling': 'yes', 'paymentmethod': 'electronic_check', 'tenure': 1, 'monthlycharges': 29.85, 'totalcharges': 29.85}
output: 0.6274453618230489


This way, we can load the model and apply it to the customer we specified in the script. 

Of course, we aren't going to manually put the information about customers in the script. In the next section, we'll cover a more practical approach where we will be putting the model into a web service. 

In [28]:
# Refer to train.py and predict.py

## 5.3 Web services: introduction to Flask
The easiest way to implement a web service in Python is to use Flask. It's quite light-weight, requires little code to get started, and hides most of the complexity of dealing with HTTP requests and responses. 

Before we put our model inside a web service, let's cover the basics of using Flask. For that, we'll create a simple function and make it available as a web service. 
* Writing a simple ping/pong app
* Querying it with `curl` and browser

Web service:
- A web service is a method used to communicate between electronic devices.
- Below are some methods in web services that we can use to satisfy our problems:
    - **GET:** A method used to retrieve files. For example, when we are searching for a cat image in google, we're actually requesting cat images with GET method.
    - **POST:** The second most common method used in web services. It enables sending data to a server to create or update a resource. For example, during a sign up process, we are submitting our name, username, password, etc. to a server that is using a web service. (Note that there is no specification on where the data goes)
    - **PUT:** Same as POST, but we are specifying where the data is going to.
    - **DELETE:** A method that is used to delete some data from the server.
- For more information, google "HTTP methods".

In [29]:
# Refer to ping.py

This is the content in the `ping.py` file
```python
# ping.py
def ping():
    return "PONG"
```

Run the program by executing this command on the terminal
```bash
ipython
```
Then, enter the following code in the Interactive Python mode:
```python
import ping

ping.ping()
```

Now, we want to turn this function into a web service and we'll use Flask for that
```bash
pip install flask
```

Decorator is just a way to add some extra functionality to our functions and this extra functionality that we're going to add will allow us to turn this function into a web service.

By putting `@app.route` on top of the function definition, we assign the `/ping` address of the web service to the `ping` function. 

```python
from flask import Flask

app = Flask('ping')  # Give an identity to your web service

# Use decorator to add Flask's functionality to our function
@app.route('/ping', methods=['GET'])  
def ping():
    return "PONG"

if __name__ == "__main__":
    # Run the code in local machine with debugging mode true and port 9696
    app.run(debug=True, host='0.0.0.0', port=9696)
```

The `run` method of `app` starts the service. We specify three parameters:
- `debug=True`: Restarts our application automatically when there are changes in the code.
- `host='0.0.0.0'`: Makes the web service public; otherwise, it won't be possible to reach it when it's hosted on a remote machine (e.g., in AWS).
- `port=9696`: The port that we use to access the application.

To start our service, execute this on the terminal:
```bash
python ping.py
```

`curl` is a special command line utility for communicating with web services. 

You can use `0.0.0.0` for localhost and then specify the port `9696`.

```bash
curl http://0.0.0.0:9696/ping
```

In [30]:
!curl http://127.0.0.1:9696/ping

PONG


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100     4  100     4    0     0   1646      0 --:--:-- --:--:-- --:--:--  2000


In [31]:
!curl http://localhost:9696/ping

PONG


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100     4  100     4    0     0     19      0 --:--:-- --:--:-- --:--:--    19


## 5.4 Serving the churn model with Flask
In this session, we talked about implementing the functionality of prediction to our churn web service and how to make it usable in development environment. 
* Wrapping the predict script into a Flask app
* Querying it with `requests` 
* Preparing for production: gunicorn
* Running it on Windows with waitress

There nay be situation where the campaign service is written in some other language, or a different team might be in charge of this project, which means we won't have the control we need to modify the code of the campaign service to load the model, and score the customers right in the service.

The typical solution for this problem is putting a model inside a web service--a small service (a microservice) that only takes care of scoring customers. 

So, we need to create a churn service--a service in Python that will serve the churn model. Given the features of a customer, it will respond with the probability of churn for this customer. For each customer, the campaign service will ask the churn service for the probability of churn, and if it's high enough, then we send a promotional email.

This gives us another advantage: separation of concerns. If the model is created by data scientists, then they can take ownership of the service and maintain it, while the other team takes care of the campaign service. 

To load the saved model, we use the code below:
```python
import pickle

with open('churn-model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)
```

To predict a value for a customer, we need a function like below:
```python
def predict_single(customer, dv, model):
    # Apply the one-hot encoding feature to the customer data
    X = dv.transform([customer]) 
    y_pred = model.predict_proba(X)[:, 1]
    return y_pred[0]
```

At last, we create the final function used for implementing the web service:
```python
# To send the customer information, we need to post its data
@app.route('/predict', methods=['POST'])
def predict():
    # Web services work best with JSON format
    customer = request.get_json()  # Access the content of a POST request

    prediction = predict_single(customer, dv, model)
    churn = prediction >= 0.5

    result = {
        # Cast numpy float type to Python native float type
        'churn_probability': float(prediction), 
        'churn': bool(churn), # Cast the value using bool method
    }
    # Send back the result in JSON format to the user
    return jsonify(result) 
```

To get a response, we post customer data as `json`:
```python
# A new customer information
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

import requests # We need the requests library to use the POST method

url = 'http://localhost:9696/predict' # The route we made for prediction
# Post the customer information in JSON format
response = requests.post(url, json=customer)
result = response.json() # Get the server response
print(result)
```

To fix the "This is a development server. Do not use it in a production deployment. Use a production WSGI server instead." warning:
* Consider creating a WSGI server using gunicorn. Use the command `pip install gunicorn` to install it. To run the WSGI server, simply execute the command `gunicorn --bind 0.0.0.0:9696 churn:app`. Note that in **churn:app**, 'churn' is the name we set for the file containing the code `app = Flask('churn')` (e.g., churn.py). You may need to change it to match the name of your Flask app file. 
* Windows users need to use an alternative library, `waitress`, because the windows system do not support some dependencies of the gunicorn library. Use the command `pip install waitress` to install it.
* To run the waitress WSGI server, use the command `waitress-serve --listen=0.0.0.0:9696 churn:app`. 
* To test it, you can run the code above and the result will be the same.

`05-predict-test.ipynb`

#### Making requests

Testing this code is a bit more difficult than previously. We need to use POST requests and include the customer we want to score in the body of the request. The simplest way of doing this is to use the requests library in Python. 

In [71]:
import requests

In [72]:
# The URL where the service lives
url = 'http://localhost:9696/predict'

In [73]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'two_year',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [74]:
customer

{'gender': 'female',
 'seniorcitizen': 0,
 'partner': 'yes',
 'dependents': 'no',
 'phoneservice': 'no',
 'multiplelines': 'no_phone_service',
 'internetservice': 'dsl',
 'onlinesecurity': 'no',
 'onlinebackup': 'yes',
 'deviceprotection': 'no',
 'techsupport': 'no',
 'streamingtv': 'no',
 'streamingmovies': 'no',
 'contract': 'two_year',
 'paperlessbilling': 'yes',
 'paymentmethod': 'electronic_check',
 'tenure': 1,
 'monthlycharges': 29.85,
 'totalcharges': 29.85}

`requests.post(url, json=customer)` will give us a TypeError: Object of type bool_ is not JSON serializable. 

We need to cast both `y_pred` and `churn` variables in `predict.py` into their respective Python datatypes because they're currently under the numpy datatypes. 

In [75]:
requests.post(url, json=customer)

<Response [200]>

In [76]:
# Sends the customer (as JSON) in the POST request and parses the response as JSON
response = requests.post(url, json=customer).json()

In [77]:
response

{'churn': False, 'churn_probability': 0.29549015063752093}

In [78]:
if response['churn'] == True:
    # 'asdx-123d' refers to the customer_id
    print('sending promo email to', 'asdx-123d')

If the campaign service used Python, this is exactly how it could communicate with the churn service and decide who should get promotional emails. 

Note: Some tools like [Postman](https://www.postman.com/), make it easier to test web services. 

In [79]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}
customer

{'gender': 'female',
 'seniorcitizen': 0,
 'partner': 'yes',
 'dependents': 'no',
 'phoneservice': 'no',
 'multiplelines': 'no_phone_service',
 'internetservice': 'dsl',
 'onlinesecurity': 'no',
 'onlinebackup': 'yes',
 'deviceprotection': 'no',
 'techsupport': 'no',
 'streamingtv': 'no',
 'streamingmovies': 'no',
 'contract': 'month-to-month',
 'paperlessbilling': 'yes',
 'paymentmethod': 'electronic_check',
 'tenure': 1,
 'monthlycharges': 29.85,
 'totalcharges': 29.85}

In [80]:
response = requests.post(url, json=customer).json()
response

{'churn': True, 'churn_probability': 0.6274453618230489}

In [81]:
if response['churn']:
    # 'xyz-123' refers to the customer_id
    print('sending email to', 'xyz-123')

sending email to xyz-123


WSGI stands for *web server gateway interface*, which is a specification describing how Python applications should handle HTTP requests.

Let's address the warning by installing a production WSGI server. We have multiple options in Python: install Gunicorn or Waitress.

We'll use Waitress because Gunicorn doesn't work on Windows: it relies on features specific to Linux and Unix (which includes MacOS). Later, we will use Docker, which will solve this problem--it runs Linux inside a container. 
```bash
pip install waitress
```

To use it
```bash
waitress-serve --listen=0.0.0.0:9696 predict:app
```

In [82]:
# Refer to `predict-test.py`

Execute the script
```bash
python predict-test.py
```

Unlike the Flask built-in web server, Gunicorn or Waitress is ready for production, so it will not have any problems under load when we start using it. 

We see that it is able to communicate with Waitress and get back a successful response. 

## 5.5 Python virtual environment: Pipenv
In this session, we're going to create a virtual environment for our project.
* Dependency and environment management
* Why we need virtual environment
* Installing Pipenv
* Installing libraries with Pipenv
* Running things with Pipenv

**What is a virtual environment and how to create one?**

Everytime we're running a file from a directory, we're using the executive files from a global directory. For example, when we install Python on our machine, the executable files running our codes will lie somewhere in */home/username/python/bin/*, while the pip command can be located in */home/username/python/bin/pip*.

Sometimes the versions of libraries conflict, resulting in massive errors for the project. An example would be an old project we had that uses sklearn library version 0.24.1, but we tried to run it using sklearn version 1.0.0 in our current environment. We may get into errors because of the version conflict.

To solve the conflict, we can create virtual environments. Virtual environment separates the libraries we installed on our system from the libraries we installed (with specified version) to run our project with. There are a lot of ways to create a virtual environment. One of them is using the `pipenv` library.

To install `pipenv`:
```bash
pip install pipenv
```

After installing `pipenv`, we need to install the libraries we need for our project in the newly created virtual environment
```bash
pip install numpy sklearn==1.5.2 flask
```

The `pipenv` command comes with two files named *Pipfile* and *Pipfile.lock*. In *Pipfile*, the libraries we installed are named. If we specified the library version, it's also specified in *Pipfile*.

In *Pipfile.lock*, we can see the names of each library with its installed version, together with a hash checksum for reproducibility if we were to move the environment to another machine.

To run the project in another machine, we can install the required libraries with the command:
```bash
pipenv install
```
> This command will look into *Pipfile* and *Pipfile.lock* to install the libraries with specified version.

After installing the required libraries, run the project in the virtual environment:
```bash
pipenv shell
```
> This will direct us to the virtual environment's shell and any command executed hereon will be using the virtual environment's libraries. 


For local development, Anaconda is a perfect tool because it has almost all the libraries we may ever need. For production, we prefer to have only the libraries we actually need. 

Additionally, different services have different requirements. Often, these requirements conflict, so we cannot use the same environment for running multiple services at the same time. 

Here, we'll learn how to manage dependencies of our application in an isolated way that doesn't interfere with other services. **Pipenv**, for managing Python libraries, and  subsequently, **Docker**, for managing the system dependencies such as the operating system and the system libraries. 

**Virtual Environment**
- virtual env /venv
- conda
- pipenv (officially recommended package management from the python community)
- poetry (considered more cool but works the same way)

Install the virtual environment
```bash
pip install pipenv
```
Install the libraries
```bash
pipenv install numpy scikit-learn==1.5.2 flask
```

Pipfile
```
[[source]]
url = "https://pypi.org/simple"
verify_ssl = true
name = "pypi"

[packages]
numpy = "*"
scikit-learn = "==1.5.2"
flask = "*"
waitress = "*"

[dev-packages]

[requires]
python_version = "3.11"
```
For scikit-learn, we specified the exact version we want for our project. 

[dev-packages]: Packages are basically packages that you only need for development so you only want them on your laptop but not when you deploy your service to production environment. 

Install gunicorn
```bash
pipenv install gunicorn
```
Or alternatively, install waitress
```bash
pipenv install waitress
```

We save everything in *Pipfile* to git and clone this repository on a different computer. For instance, our colleague wants to start working on this project. What they do after cloning is they just do `pipenv install`. And that's all they need to do. They don't need to write all the names of the libraries when we have the *Pipfile* and *Pipfile.lock* to assist in figuring out which dependencies it needs to install. 

Pipfile.lock
```
{
    "_meta": {
        "hash": {
            "sha256": "7232eab033193e7e2c0cb5c2a9edcb12006a132b022d34b415000bc49a10b3de"
        },
        "pipfile-spec": 6,
        "requires": {
            "python_version": "3.11"
        },
        "sources": [
            {
                "name": "pypi",
                "url": "https://pypi.org/simple",
                "verify_ssl": true
            }
        ]
    },
    "default": {
        "blinker": {
            "hashes": [
                "sha256:1779309f71bf239144b9399d06ae925637cf6634cf6bd131104184531bf67c01",
                "sha256:8f77b09d3bf7c795e969e9486f39c2c5e9c39d4ee07424be2bc594ece9642d83"
            ],
            "markers": "python_version >= '3.8'",
            "version": "==1.8.2"
        },
        "click": {
            "hashes": [
                "sha256:ae74fb96c20a0277a1d615f1e4d73c8414f5a98db8b799a7931d1582f3390c28",
                "sha256:ca9853ad459e787e2192211578cc907e7594e294c7ccc834310722b41b9ca6de"
            ],
            "markers": "python_version >= '3.7'",
            "version": "==8.1.7"
        },
        "colorama": {
            "hashes": [
                "sha256:08695f5cb7ed6e0531a20572697297273c47b8cae5a63ffc6d6ed5c201be6e44",
                "sha256:4f1d9991f5acc0ca119f9d443620b77f9d6b33703e51011c16baf57afb285fc6"
            ],
            "markers": "python_version >= '2.7' and python_version not in '3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6'",
            "version": "==0.4.6"
        },
        "flask": {
            "hashes": [
                "sha256:34e815dfaa43340d1d15a5c3a02b8476004037eb4840b34910c6e21679d288f3",
                "sha256:ceb27b0af3823ea2737928a4d99d125a06175b8512c445cbd9a9ce200ef76842"
            ],
            "index": "pypi",
            "markers": "python_version >= '3.8'",
            "version": "==3.0.3"
        },
        "itsdangerous": {
            "hashes": [
                "sha256:c6242fc49e35958c8b15141343aa660db5fc54d4f13a1db01a3f5891b98700ef",
                "sha256:e0050c0b7da1eea53ffaf149c0cfbb5c6e2e2b69c4bef22c81fa6eb73e5f6173"
            ],
            "markers": "python_version >= '3.8'",
            "version": "==2.2.0"
        },
        "jinja2": {
            "hashes": [
                "sha256:4a3aee7acbbe7303aede8e9648d13b8bf88a429282aa6122a993f0ac800cb369",
                "sha256:bc5dd2abb727a5319567b7a813e6a2e7318c39f4f487cfe6c89c6f9c7d25197d"
            ],
            "markers": "python_version >= '3.7'",
            "version": "==3.1.4"
        },
        "joblib": {
            "hashes": [
                "sha256:06d478d5674cbc267e7496a410ee875abd68e4340feff4490bcb7afb88060ae6",
                "sha256:2382c5816b2636fbd20a09e0f4e9dad4736765fdfb7dca582943b9c1366b3f0e"
            ],
            "markers": "python_version >= '3.8'",
            "version": "==1.4.2"
        },
        "markupsafe": {
            "hashes": [
                "sha256:0bff5e0ae4ef2e1ae4fdf2dfd5b76c75e5c2fa4132d05fc1b0dabcd20c7e28c4",
                "sha256:0f4ca02bea9a23221c0182836703cbf8930c5e9454bacce27e767509fa286a30",
                "sha256:1225beacc926f536dc82e45f8a4d68502949dc67eea90eab715dea3a21c1b5f0",
                "sha256:131a3c7689c85f5ad20f9f6fb1b866f402c445b220c19fe4308c0b147ccd2ad9",
                "sha256:15ab75ef81add55874e7ab7055e9c397312385bd9ced94920f2802310c930396",
                "sha256:1a9d3f5f0901fdec14d8d2f66ef7d035f2157240a433441719ac9a3fba440b13",
                "sha256:1c99d261bd2d5f6b59325c92c73df481e05e57f19837bdca8413b9eac4bd8028",
                "sha256:1e084f686b92e5b83186b07e8a17fc09e38fff551f3602b249881fec658d3eca",
                "sha256:2181e67807fc2fa785d0592dc2d6206c019b9502410671cc905d132a92866557",
                "sha256:2cb8438c3cbb25e220c2ab33bb226559e7afb3baec11c4f218ffa7308603c832",
                "sha256:3169b1eefae027567d1ce6ee7cae382c57fe26e82775f460f0b2778beaad66c0",
                "sha256:3809ede931876f5b2ec92eef964286840ed3540dadf803dd570c3b7e13141a3b",
                "sha256:38a9ef736c01fccdd6600705b09dc574584b89bea478200c5fbf112a6b0d5579",
                "sha256:3d79d162e7be8f996986c064d1c7c817f6df3a77fe3d6859f6f9e7be4b8c213a",
                "sha256:444dcda765c8a838eaae23112db52f1efaf750daddb2d9ca300bcae1039adc5c",
                "sha256:48032821bbdf20f5799ff537c7ac3d1fba0ba032cfc06194faffa8cda8b560ff",
                "sha256:4aa4e5faecf353ed117801a068ebab7b7e09ffb6e1d5e412dc852e0da018126c",
                "sha256:52305740fe773d09cffb16f8ed0427942901f00adedac82ec8b67752f58a1b22",
                "sha256:569511d3b58c8791ab4c2e1285575265991e6d8f8700c7be0e88f86cb0672094",
                "sha256:57cb5a3cf367aeb1d316576250f65edec5bb3be939e9247ae594b4bcbc317dfb",
                "sha256:5b02fb34468b6aaa40dfc198d813a641e3a63b98c2b05a16b9f80b7ec314185e",
                "sha256:6381026f158fdb7c72a168278597a5e3a5222e83ea18f543112b2662a9b699c5",
                "sha256:6af100e168aa82a50e186c82875a5893c5597a0c1ccdb0d8b40240b1f28b969a",
                "sha256:6c89876f41da747c8d3677a2b540fb32ef5715f97b66eeb0c6b66f5e3ef6f59d",
                "sha256:6e296a513ca3d94054c2c881cc913116e90fd030ad1c656b3869762b754f5f8a",
                "sha256:70a87b411535ccad5ef2f1df5136506a10775d267e197e4cf531ced10537bd6b",
                "sha256:7e94c425039cde14257288fd61dcfb01963e658efbc0ff54f5306b06054700f8",
                "sha256:846ade7b71e3536c4e56b386c2a47adf5741d2d8b94ec9dc3e92e5e1ee1e2225",
                "sha256:88416bd1e65dcea10bc7569faacb2c20ce071dd1f87539ca2ab364bf6231393c",
                "sha256:88b49a3b9ff31e19998750c38e030fc7bb937398b1f78cfa599aaef92d693144",
                "sha256:8c4e8c3ce11e1f92f6536ff07154f9d49677ebaaafc32db9db4620bc11ed480f",
                "sha256:8e06879fc22a25ca47312fbe7c8264eb0b662f6db27cb2d3bbbc74b1df4b9b87",
                "sha256:9025b4018f3a1314059769c7bf15441064b2207cb3f065e6ea1e7359cb46db9d",
                "sha256:93335ca3812df2f366e80509ae119189886b0f3c2b81325d39efdb84a1e2ae93",
                "sha256:9778bd8ab0a994ebf6f84c2b949e65736d5575320a17ae8984a77fab08db94cf",
                "sha256:9e2d922824181480953426608b81967de705c3cef4d1af983af849d7bd619158",
                "sha256:a123e330ef0853c6e822384873bef7507557d8e4a082961e1defa947aa59ba84",
                "sha256:a904af0a6162c73e3edcb969eeeb53a63ceeb5d8cf642fade7d39e7963a22ddb",
                "sha256:ad10d3ded218f1039f11a75f8091880239651b52e9bb592ca27de44eed242a48",
                "sha256:b424c77b206d63d500bcb69fa55ed8d0e6a3774056bdc4839fc9298a7edca171",
                "sha256:b5a6b3ada725cea8a5e634536b1b01c30bcdcd7f9c6fff4151548d5bf6b3a36c",
                "sha256:ba8062ed2cf21c07a9e295d5b8a2a5ce678b913b45fdf68c32d95d6c1291e0b6",
                "sha256:ba9527cdd4c926ed0760bc301f6728ef34d841f405abf9d4f959c478421e4efd",
                "sha256:bbcb445fa71794da8f178f0f6d66789a28d7319071af7a496d4d507ed566270d",
                "sha256:bcf3e58998965654fdaff38e58584d8937aa3096ab5354d493c77d1fdd66d7a1",
                "sha256:c0ef13eaeee5b615fb07c9a7dadb38eac06a0608b41570d8ade51c56539e509d",
                "sha256:cabc348d87e913db6ab4aa100f01b08f481097838bdddf7c7a84b7575b7309ca",
                "sha256:cdb82a876c47801bb54a690c5ae105a46b392ac6099881cdfb9f6e95e4014c6a",
                "sha256:cfad01eed2c2e0c01fd0ecd2ef42c492f7f93902e39a42fc9ee1692961443a29",
                "sha256:d16a81a06776313e817c951135cf7340a3e91e8c1ff2fac444cfd75fffa04afe",
                "sha256:d8213e09c917a951de9d09ecee036d5c7d36cb6cb7dbaece4c71a60d79fb9798",
                "sha256:e07c3764494e3776c602c1e78e298937c3315ccc9043ead7e685b7f2b8d47b3c",
                "sha256:e17c96c14e19278594aa4841ec148115f9c7615a47382ecb6b82bd8fea3ab0c8",
                "sha256:e444a31f8db13eb18ada366ab3cf45fd4b31e4db1236a4448f68778c1d1a5a2f",
                "sha256:e6a2a455bd412959b57a172ce6328d2dd1f01cb2135efda2e4576e8a23fa3b0f",
                "sha256:eaa0a10b7f72326f1372a713e73c3f739b524b3af41feb43e4921cb529f5929a",
                "sha256:eb7972a85c54febfb25b5c4b4f3af4dcc731994c7da0d8a0b4a6eb0640e1d178",
                "sha256:ee55d3edf80167e48ea11a923c7386f4669df67d7994554387f84e7d8b0a2bf0",
                "sha256:f3818cb119498c0678015754eba762e0d61e5b52d34c8b13d770f0719f7b1d79",
                "sha256:f8b3d067f2e40fe93e1ccdd6b2e1d16c43140e76f02fb1319a05cf2b79d99430",
                "sha256:fcabf5ff6eea076f859677f5f0b6b5c1a51e70a376b0579e0eadef8db48c6b50"
            ],
            "markers": "python_version >= '3.9'",
            "version": "==3.0.2"
        },
        "numpy": {
            "hashes": [
                "sha256:05b2d4e667895cc55e3ff2b56077e4c8a5604361fc21a042845ea3ad67465aa8",
                "sha256:12edb90831ff481f7ef5f6bc6431a9d74dc0e5ff401559a71e5e4611d4f2d466",
                "sha256:13311c2db4c5f7609b462bc0f43d3c465424d25c626d95040f073e30f7570e35",
                "sha256:13532a088217fa624c99b843eeb54640de23b3414b14aa66d023805eb731066c",
                "sha256:13602b3174432a35b16c4cfb5de9a12d229727c3dd47a6ce35111f2ebdf66ff4",
                "sha256:1600068c262af1ca9580a527d43dc9d959b0b1d8e56f8a05d830eea39b7c8af6",
                "sha256:1b8cde4f11f0a975d1fd59373b32e2f5a562ade7cde4f85b7137f3de8fbb29a0",
                "sha256:1c193d0b0238638e6fc5f10f1b074a6993cb13b0b431f64079a509d63d3aa8b7",
                "sha256:1ebec5fd716c5a5b3d8dfcc439be82a8407b7b24b230d0ad28a81b61c2f4659a",
                "sha256:242b39d00e4944431a3cd2db2f5377e15b5785920421993770cddb89992c3f3a",
                "sha256:259ec80d54999cc34cd1eb8ded513cb053c3bf4829152a2e00de2371bd406f5e",
                "sha256:2abbf905a0b568706391ec6fa15161fad0fb5d8b68d73c461b3c1bab6064dd62",
                "sha256:2cbba4b30bf31ddbe97f1c7205ef976909a93a66bb1583e983adbd155ba72ac2",
                "sha256:2ffef621c14ebb0188a8633348504a35c13680d6da93ab5cb86f4e54b7e922b5",
                "sha256:30d53720b726ec36a7f88dc873f0eec8447fbc93d93a8f079dfac2629598d6ee",
                "sha256:32e16a03138cabe0cb28e1007ee82264296ac0983714094380b408097a418cfe",
                "sha256:43cca367bf94a14aca50b89e9bc2061683116cfe864e56740e083392f533ce7a",
                "sha256:456e3b11cb79ac9946c822a56346ec80275eaf2950314b249b512896c0d2505e",
                "sha256:4d6ec0d4222e8ffdab1744da2560f07856421b367928026fb540e1945f2eeeaf",
                "sha256:5006b13a06e0b38d561fab5ccc37581f23c9511879be7693bd33c7cd15ca227c",
                "sha256:675c741d4739af2dc20cd6c6a5c4b7355c728167845e3c6b0e824e4e5d36a6c3",
                "sha256:6cdb606a7478f9ad91c6283e238544451e3a95f30fb5467fbf715964341a8a86",
                "sha256:6d95f286b8244b3649b477ac066c6906fbb2905f8ac19b170e2175d3d799f4df",
                "sha256:76322dcdb16fccf2ac56f99048af32259dcc488d9b7e25b51e5eca5147a3fb98",
                "sha256:7c1c60328bd964b53f8b835df69ae8198659e2b9302ff9ebb7de4e5a5994db3d",
                "sha256:860ec6e63e2c5c2ee5e9121808145c7bf86c96cca9ad396c0bd3e0f2798ccbe2",
                "sha256:8e00ea6fc82e8a804433d3e9cedaa1051a1422cb6e443011590c14d2dea59146",
                "sha256:9c6c754df29ce6a89ed23afb25550d1c2d5fdb9901d9c67a16e0b16eaf7e2550",
                "sha256:a26ae94658d3ba3781d5e103ac07a876b3e9b29db53f68ed7df432fd033358a8",
                "sha256:a65acfdb9c6ebb8368490dbafe83c03c7e277b37e6857f0caeadbbc56e12f4fb",
                "sha256:a7d80b2e904faa63068ead63107189164ca443b42dd1930299e0d1cb041cec2e",
                "sha256:a84498e0d0a1174f2b3ed769b67b656aa5460c92c9554039e11f20a05650f00d",
                "sha256:ab4754d432e3ac42d33a269c8567413bdb541689b02d93788af4131018cbf366",
                "sha256:ad369ed238b1959dfbade9018a740fb9392c5ac4f9b5173f420bd4f37ba1f7a0",
                "sha256:b1d0fcae4f0949f215d4632be684a539859b295e2d0cb14f78ec231915d644db",
                "sha256:b42a1a511c81cc78cbc4539675713bbcf9d9c3913386243ceff0e9429ca892fe",
                "sha256:bd33f82e95ba7ad632bc57837ee99dba3d7e006536200c4e9124089e1bf42426",
                "sha256:bdd407c40483463898b84490770199d5714dcc9dd9b792f6c6caccc523c00952",
                "sha256:c6eef7a2dbd0abfb0d9eaf78b73017dbfd0b54051102ff4e6a7b2980d5ac1a03",
                "sha256:c82af4b2ddd2ee72d1fc0c6695048d457e00b3582ccde72d8a1c991b808bb20f",
                "sha256:d666cb72687559689e9906197e3bec7b736764df6a2e58ee265e360663e9baf7",
                "sha256:d7bf0a4f9f15b32b5ba53147369e94296f5fffb783db5aacc1be15b4bf72f43b",
                "sha256:d82075752f40c0ddf57e6e02673a17f6cb0f8eb3f587f63ca1eaab5594da5b17",
                "sha256:da65fb46d4cbb75cb417cddf6ba5e7582eb7bb0b47db4b99c9fe5787ce5d91f5",
                "sha256:e2b49c3c0804e8ecb05d59af8386ec2f74877f7ca8fd9c1e00be2672e4d399b1",
                "sha256:e585c8ae871fd38ac50598f4763d73ec5497b0de9a0ab4ef5b69f01c6a046142",
                "sha256:e8d3ca0a72dd8846eb6f7dfe8f19088060fcb76931ed592d29128e0219652884",
                "sha256:ef444c57d664d35cac4e18c298c47d7b504c66b17c2ea91312e979fcfbdfb08a",
                "sha256:f1eb068ead09f4994dec71c24b2844f1e4e4e013b9629f812f292f04bd1510d9",
                "sha256:f2ded8d9b6f68cc26f8425eda5d3877b47343e68ca23d0d0846f4d312ecaa445",
                "sha256:f751ed0a2f250541e19dfca9f1eafa31a392c71c832b6bb9e113b10d050cb0f1",
                "sha256:faa88bc527d0f097abdc2c663cddf37c05a1c2f113716601555249805cf573f1",
                "sha256:fc44e3c68ff00fd991b59092a54350e6e4911152682b4782f68070985aa9e648"
            ],
            "index": "pypi",
            "markers": "python_version >= '3.10'",
            "version": "==2.1.2"
        },
        "scikit-learn": {
            "hashes": [
                "sha256:03b6158efa3faaf1feea3faa884c840ebd61b6484167c711548fce208ea09445",
                "sha256:178ddd0a5cb0044464fc1bfc4cca5b1833bfc7bb022d70b05db8530da4bb3dd3",
                "sha256:1ff45e26928d3b4eb767a8f14a9a6efbf1cbff7c05d1fb0f95f211a89fd4f5de",
                "sha256:299406827fb9a4f862626d0fe6c122f5f87f8910b86fe5daa4c32dcd742139b6",
                "sha256:2d4cad1119c77930b235579ad0dc25e65c917e756fe80cab96aa3b9428bd3fb0",
                "sha256:394397841449853c2290a32050382edaec3da89e35b3e03d6cc966aebc6a8ae6",
                "sha256:3a686885a4b3818d9e62904d91b57fa757fc2bed3e465c8b177be652f4dd37c8",
                "sha256:3b923d119d65b7bd555c73be5423bf06c0105678ce7e1f558cb4b40b0a5502b1",
                "sha256:3bed4909ba187aca80580fe2ef370d9180dcf18e621a27c4cf2ef10d279a7efe",
                "sha256:52788f48b5d8bca5c0736c175fa6bdaab2ef00a8f536cda698db61bd89c551c1",
                "sha256:57cc1786cfd6bd118220a92ede80270132aa353647684efa385a74244a41e3b1",
                "sha256:643964678f4b5fbdc95cbf8aec638acc7aa70f5f79ee2cdad1eec3df4ba6ead8",
                "sha256:6c16d84a0d45e4894832b3c4d0bf73050939e21b99b01b6fd59cbb0cf39163b6",
                "sha256:757c7d514ddb00ae249832fe87100d9c73c6ea91423802872d9e74970a0e40b9",
                "sha256:8c412ccc2ad9bf3755915e3908e677b367ebc8d010acbb3f182814524f2e5540",
                "sha256:b0768ad641981f5d3a198430a1d31c3e044ed2e8a6f22166b4d546a5116d7908",
                "sha256:b4237ed7b3fdd0a4882792e68ef2545d5baa50aca3bb45aa7df468138ad8f94d",
                "sha256:b7b0f9a0b1040830d38c39b91b3a44e1b643f4b36e36567b80b7c6bd2202a27f",
                "sha256:c15b1ca23d7c5f33cc2cb0a0d6aaacf893792271cddff0edbd6a40e8319bc113",
                "sha256:ca64b3089a6d9b9363cd3546f8978229dcbb737aceb2c12144ee3f70f95684b7",
                "sha256:e9a702e2de732bbb20d3bad29ebd77fc05a6b427dc49964300340e4c9328b3f5",
                "sha256:f60021ec1574e56632be2a36b946f8143bf4e5e6af4a06d85281adc22938e0dd",
                "sha256:f7284ade780084d94505632241bf78c44ab3b6f1e8ccab3d2af58e0e950f9c12",
                "sha256:f763897fe92d0e903aa4847b0aec0e68cadfff77e8a0687cabd946c89d17e675",
                "sha256:f8b0ccd4a902836493e026c03256e8b206656f91fbcc4fde28c57a5b752561f1",
                "sha256:f932a02c3f4956dfb981391ab24bda1dbd90fe3d628e4b42caef3e041c67707a"
            ],
            "index": "pypi",
            "markers": "python_version >= '3.9'",
            "version": "==1.5.2"
        },
        "scipy": {
            "hashes": [
                "sha256:0c2f95de3b04e26f5f3ad5bb05e74ba7f68b837133a4492414b3afd79dfe540e",
                "sha256:1729560c906963fc8389f6aac023739ff3983e727b1a4d87696b7bf108316a79",
                "sha256:278266012eb69f4a720827bdd2dc54b2271c97d84255b2faaa8f161a158c3b37",
                "sha256:2843f2d527d9eebec9a43e6b406fb7266f3af25a751aa91d62ff416f54170bc5",
                "sha256:2da0469a4ef0ecd3693761acbdc20f2fdeafb69e6819cc081308cc978153c675",
                "sha256:2ff0a7e01e422c15739ecd64432743cf7aae2b03f3084288f399affcefe5222d",
                "sha256:2ff38e22128e6c03ff73b6bb0f85f897d2362f8c052e3b8ad00532198fbdae3f",
                "sha256:30ac8812c1d2aab7131a79ba62933a2a76f582d5dbbc695192453dae67ad6310",
                "sha256:3a1b111fac6baec1c1d92f27e76511c9e7218f1695d61b59e05e0fe04dc59617",
                "sha256:4079b90df244709e675cdc8b93bfd8a395d59af40b72e339c2287c91860deb8e",
                "sha256:5149e3fd2d686e42144a093b206aef01932a0059c2a33ddfa67f5f035bdfe13e",
                "sha256:5a275584e726026a5699459aa72f828a610821006228e841b94275c4a7c08417",
                "sha256:631f07b3734d34aced009aaf6fedfd0eb3498a97e581c3b1e5f14a04164a456d",
                "sha256:716e389b694c4bb564b4fc0c51bc84d381735e0d39d3f26ec1af2556ec6aad94",
                "sha256:8426251ad1e4ad903a4514712d2fa8fdd5382c978010d1c6f5f37ef286a713ad",
                "sha256:8475230e55549ab3f207bff11ebfc91c805dc3463ef62eda3ccf593254524ce8",
                "sha256:8bddf15838ba768bb5f5083c1ea012d64c9a444e16192762bd858f1e126196d0",
                "sha256:8e32dced201274bf96899e6491d9ba3e9a5f6b336708656466ad0522d8528f69",
                "sha256:8f9ea80f2e65bdaa0b7627fb00cbeb2daf163caa015e59b7516395fe3bd1e066",
                "sha256:97c5dddd5932bd2a1a31c927ba5e1463a53b87ca96b5c9bdf5dfd6096e27efc3",
                "sha256:a49f6ed96f83966f576b33a44257d869756df6cf1ef4934f59dd58b25e0327e5",
                "sha256:af29a935803cc707ab2ed7791c44288a682f9c8107bc00f0eccc4f92c08d6e07",
                "sha256:b05d43735bb2f07d689f56f7b474788a13ed8adc484a85aa65c0fd931cf9ccd2",
                "sha256:b28d2ca4add7ac16ae8bb6632a3c86e4b9e4d52d3e34267f6e1b0c1f8d87e389",
                "sha256:b99722ea48b7ea25e8e015e8341ae74624f72e5f21fc2abd45f3a93266de4c5d",
                "sha256:baff393942b550823bfce952bb62270ee17504d02a1801d7fd0719534dfb9c84",
                "sha256:c0ee987efa6737242745f347835da2cc5bb9f1b42996a4d97d5c7ff7928cb6f2",
                "sha256:d0d2821003174de06b69e58cef2316a6622b60ee613121199cb2852a873f8cf3",
                "sha256:e0cf28db0f24a38b2a0ca33a85a54852586e43cf6fd876365c86e0657cfe7d73",
                "sha256:e4f5a7c49323533f9103d4dacf4e4f07078f360743dec7f7596949149efeec06",
                "sha256:eb58ca0abd96911932f688528977858681a59d61a7ce908ffd355957f7025cfc",
                "sha256:edaf02b82cd7639db00dbff629995ef185c8df4c3ffa71a5562a595765a06ce1",
                "sha256:fef8c87f8abfb884dac04e97824b61299880c43f4ce675dd2cbeadd3c9b466d2"
            ],
            "markers": "python_version >= '3.10'",
            "version": "==1.14.1"
        },
        "threadpoolctl": {
            "hashes": [
                "sha256:082433502dd922bf738de0d8bcc4fdcbf0979ff44c42bd40f5af8a282f6fa107",
                "sha256:56c1e26c150397e58c4926da8eeee87533b1e32bef131bd4bf6a2f45f3185467"
            ],
            "markers": "python_version >= '3.8'",
            "version": "==3.5.0"
        },
        "waitress": {
            "hashes": [
                "sha256:26cdbc593093a15119351690752c99adc13cbc6786d75f7b6341d1234a3730ac",
                "sha256:ef0c1f020d9f12a515c4ec65c07920a702613afcad1dbfdc3bcec256b6c072b3"
            ],
            "index": "pypi",
            "markers": "python_full_version >= '3.9.0'",
            "version": "==3.0.1"
        },
        "werkzeug": {
            "hashes": [
                "sha256:1bc0c2310d2fbb07b1dd1105eba2f7af72f322e1e455f2f93c993bee8c8a5f17",
                "sha256:a8dd59d4de28ca70471a34cba79bed5f7ef2e036a76b3ab0835474246eb41f8d"
            ],
            "markers": "python_version >= '3.8'",
            "version": "==3.0.6"
        }
    },
    "develop": {}
}
```
*Pipfile.lock* is a big JSON file and it contains exact versions for every dependencies we have. 

Using `flask` as an example, it specified the exact version of flask that was used for this project and the 'hashes' variable contains the checksum for the files to avoid any surprises. It kind of pins the versions and when you do `pipenv install`, it make sure that these exact versions are used when you create this virtual environment on a different computer. This helps with reproducibility: making sure that the dependencies you have on your machine will be exactly the same when your colleague tries to install it to avoid some nasty bugs with personal compatibility. 

Always pin your dependencies so that when a new version that changes something in the behavior gets rolled out, you're able to avoid such surprises. 

Run our service (to get into the virtual environment we've created)
```bash
pipenv shell
```
Let's understand how virtual environment makes this happen:
```bash
ls
which gunicorn
echo $PATH
```
It does so by adding the virtual environment at the beginning of our path variable and that's why when looking for gunicorn, it goes to the virtualenvs folder first. 

Use gunicorn from this virtual environment that we've created
```bash
gunicorn --bind 0.0.0.0:9696 predict:app
```
Or alternatively, use waitress from this virtual environment that we've created 
```bash
waitress-serve --listen=0.0.0.0:9696 predict:app
```

To exit this shell (virtual environment), execute this command
```bash
exit
```
And now we're back to the environment before entering the virtual environment. 

To get back into the virtual environment, we need to run `pipenv shell` again. 

Alternatively, we can use 
```bash
pipenv run waitress-serve --listen=0.0.0.0:9696 predict:app
```
to execute, `pipenv shell` + `waitress-serve --listen=0.0.0.0:9696 predict:app` using a single command, inside the virtual environment immediately. 

In summary, **Pipenv** is a great tool for managing dependencies: it isolates the required libraries into a separate environment, thus helping us avoid conflicts between different versions of the same package. 

## 5.6 Environment management: Docker
We've learned how to manage Python dependencies with Pipenv. However, some of the dependencies live outside of Python. Most importantly, these dependencies include the operating system (OS) as well as the system libraries. 

For example, we might use Ubuntu version 16.04 for developing our service, but if some of our colleagues use Ubuntu version 20.04, they may run into trouble when trying to execute the service on their laptop.

Docker solves this "but it works on my machine" problem by also packaging the OS and the system libraries into a *Docker container*--a self-contained environment that works anywhere where Docker is installed. 
* Why we need Docker
* Running a Python image with docker
* Dockerfile
* Building a docker image
* Running a docker image

#### Getting started with Docker

**Ubuntu**
```bash
sudo apt-get install docker.io
```

**Windows**

Follow the instruction by Andrew Lock in this link: https://andrewlock.net/installing-docker-desktop-for-windows/

**MacOS**

Follow the steps in the [Docker docs](https://docs.docker.com/desktop/install/mac-install/)

Packing our project into a Docker container enables us to run our project on the host machine--our laptop (regardless of the OS) or any public cloud provider. 

First, we need to create a Docker image. The Docker image is a description of our service that includes all the settings and project dependencies. Go over [Docker website](https://hub.docker.com/search?type=image) to search for a suitable base Docker image to use for our project. Docker will later use the image to create a container as specified in the Dockerfile.

Here's a Dockerfile

(Note: There should be no comments in Dockerfile, so remove the comments when you copy)
```docker
# First install the python 3.8, the slim version uses less space
FROM python:3.8.12-slim

# Install pipenv library in Docker 
RUN pip install pipenv

# create a directory in Docker named app and we're using it as work directory 
WORKDIR /app                                                                

# Copy the Pip files into our working derectory 
COPY ["Pipfile", "Pipfile.lock", "./"]

# install the pipenv dependencies for the project and deploy them.
RUN pipenv install --deploy --system

# Copy any python files and the model we had to the working directory of Docker 
COPY ["*.py", "churn-model.bin", "./"]

# We need to expose the 9696 port because we're not able to communicate with Docker outside it
EXPOSE 9696

# If we run the Docker image, we want our churn app to be running
ENTRYPOINT ["gunicorn", "--bind", "0.0.0.0:9696", "churn_serving:app"]
```

The flags `--deploy` and `--system` ensure that we install the dependencies directly inside the Docker container without creating an additional virtual environment (which `pipenv` does by default). 

If we don't put the last line `ENTRYPOINT`, we will be in a python shell.

Note that for the entrypoint, we put our commands in double quotes.

After creating the Dockerfile, we need to build it:

```bash
docker build -t churn-prediction .
```

To run it,  execute the command below:

```bash
docker run -it -p 9696:9696 churn-prediction:latest
```

Flag explanations: 

- `-t`: is used for specifying the tag name "churn-prediction".
- `-it`: in order for Docker to allow us access to the terminal.
- `--rm`: allows us to remove the image from the system after we're done.  
- `-p`: to map the 9696 port of the Docker to 9696 port of our machine. (first 9696 is the port number of our machine and the last one is Docker container port.)
- `--entrypoint=bash`: After running Docker, we will now be able to communicate with the container using bash (as you would normally do with the Terminal). Default is `python`.


Use `python 3.11.10-slim` as the base Docker image. `slim` means that this image is optimized (small in size). 

```bash
docker run -it --rm python:3.11.10-slim
```

Check that Python has been successful installed
```python
print('hello world')
```

To get inside this image (access its usual linux terminal), we need to overwrite the entry point with `bash`. Entry point is the default command that is executed when we do `docker run`. 
```bash
docker run -it --rm --entrypoint=bash python:3.11.10-slim
```

Get updates
```bash
apt-get update
```

Install wget
```bash
apt-get install wget
```
Whatever we do here stays here: it has no effects on the outside system of the host machine. This is good if let's say our service is doing something funny, only the docker container will be affected. 

Install pipenv
```bash
pip install pipenv
```

Everything that we want to do inside this docker image can be defined in the Dockerfile.

Build this docker image
```bash
docker build -t zoomcamp-test .
```

Run this image
```bash
docker run -it --rm --entrypoint=bash zoomcamp-test
```

Create a virtual environment and install the project dependencies
```bash
pipenv install
```

However, we don't need to create a virtual environment inside Docker because Docker is already isolated. In this Docker container, there is nothing else but our service so we don't really need this virtual environment. We want to skip creating the virtual environment and for that, we use a special key `--system --deploy`. 
```bash
RUN pipenv install --system --deploy
```

Rebuild and run the Docker image again
```bash
docker build -t zoomcamp-test .\05-deployment\
docker run -it --rm --entrypoint=bash zoomcamp-test
```

Install gunicorn
```bash
pip install gunicorn
```

Run gunicorn
```bash
gunicorn --bind=0.0.0.0:9696 predict:app
```

Result on our terminal
```text
[2024-10-31 12:38:53 +0000] [18] [INFO] Starting gunicorn 23.0.0
[2024-10-31 12:38:53 +0000] [18] [INFO] Listening at: http://0.0.0.0:9696 (18)
[2024-10-31 12:38:53 +0000] [18] [INFO] Using worker: sync
[2024-10-31 12:38:53 +0000] [19] [INFO] Booting worker with pid: 19
```
We cannot yet access this port because we need to expose this port by telling Docker that we want this port to be open to the host machine or other scripts that uses the requests library to communicate with our service in the Docker container. It needs to access the docker container through the port and this is done by port mapping (mapping the port we have on the container to the port we have on our host machine).

Finally, we need to specify the entry point in our dockerfile.

Dockerfile
```dockerfile
FROM python:3.11.10-slim

RUN pip install pipenv
RUN pip install gunicorn

WORKDIR /app
COPY ["Pipfile", "Pipfile.lock", "./"]

RUN pipenv install --system --deploy

COPY ["predict.py", "model_C=1.0.bin", "./"]

EXPOSE 9696

ENTRYPOINT ["gunicorn", "--bind=0.0.0.0:9696", "predict:app"]
```


Rebuild and run the Docker image again
```bash
# docker run -it --rm -p container_port:host_port zoomcamp-test
docker run -it --rm -p 9696:9696 zoomcamp-test
```

Execute the `predict-test.py` script again
```bash
python predict-test.py
```
We see that our script can reach the service we have in the Docker container.






In summary, Docker makes it easy to run services in a reproducible way. With Docker, the environment inside the container always stays the same. This means that if we can run our service on a laptop, it will work anywhere else. 

## 5.7 Deployment to the cloud: AWS Elastic Beanstalk (optional)

* Installing the eb cli
* Running eb locally
* Deploying the model

## 5.8 Summary

* Save models with picke
* Use Flask to turn the model into a web service
* Use a dependency & env manager
* Package it in Docker
* Deploy to the cloud

## 5.9 Explore more

* Flask is not the only framework for creating web services. Try others, e.g. FastAPI
* Experiment with other ways of managing environment, e.g. virtual env, conda, poetry.
* Explore other ways of deploying web services, e.g. GCP, Azure, Heroku, Python Anywhere, etc